In [ ]:
import os
import time

from functions.input2dataframe import *

from functions.analyse_df_pcap import *
from functions.tcpdump_filter_anon import *

from functions.analyse_df_nfdump import *
from functions.nfdump_filter_anon import *

In [ ]:
def ddos_analysis_fingerprint_filter_sanitization(input_file, debug=True):
    
    start = time.clock()
    if debug: print 'Converting input file to dataframe (time:',  
    df = input2dataframe(input_file) #input2dataframe checks the input_file type (pcap, pcapng, nfdump) and calls the right converter (e.g. pcap2dataframe, nfdump2dataframe)
#     display(df[df['dst_ip']=='185.188.206.99']['dst_port'].value_counts())
    if debug: print str(time.clock() - start)+')'
    
    #########################################################        
    #########################################################
    start = time.clock()
    if debug: print 'Analysing the dataframe for finding attack patterns...'    
        
    if 'tuple' in str(type(df)):
        all_patterns = analyse_df_nfdump(df, debug)
        if debug: print 'Dataframe completely analysed and all attack patterns identified! (time:'+str(time.clock() - start)+')'
        #########################################################        
        #########################################################
        start = time.clock()
        if debug: print 'Generating attack vectors (with the same format as the input file) and anonymizing the victim information (dst_ip) (time:' 
        for pattern in all_patterns["patterns"]:
            print pattern['start_timestamp']
            nfdump_filter_anon(input_file,  pattern, all_patterns['dst_ip'])
        #########################################################        
        #########################################################   
    else:
        all_patterns = analyse_df_pcap(df, debug=True)
        display(all_patterns)
        
        if debug: print 'Dataframe completely analysed and all attack patterns identified! (time:'+str(time.clock() - start)+')'
        #########################################################        
        #########################################################
        start = time.clock()
        if debug: print 'Generating attack vectors (same format as input file) and anonymizing victim\'s information (dst_ip) (time:' 
        for pattern in all_patterns["patterns"]:
            tcpdump_filter_anon(input_file, pattern, all_patterns["dst_ip"])
            
    if debug: print str(time.clock() - start)+')'
#     #########################################################        
#     #########################################################
#     #TODO: Generate rule/signatures for specific detection and mitigation technologies (SIDS: BRO, SNORT, SURICATA; Firewall)
    

## Execution from Jupyter Notebook

In [ ]:
input_file="input4test/prod-flow-anon-008.nfdump"
ddos_analysis_fingerprint_filter_sanitization(input_file)

## Main for python script 

In [ ]:
if __name__ == '__main__':
    import argparse
    import os.path
    
    parser = argparse.ArgumentParser(description='')
    
    parser.add_argument('--input', metavar='input_file', required=True,
                        help='Path of a input file')
    args = parser.parse_args()
    
    input_file=args.input
    
    if os.path.isfile(input_file) == False:
        print 'We were unable to find the file. Please check the file path!!'
    
        ddos_analysis_fingerprint_filter_sanitization(input_file, debug=True)